# Database I/O
([Back to Overview](../index.html#/0/3))

In [1]:
data_directory = joinpath(@__DIR__, "..", "..", "..", "exercises", "fridge")

"/Users/blaschke/Developer/hpc-julia/docs/julia for data science/01_data/../../../exercises/fridge"

In [2]:
using Dates
using TimeZones

In [3]:
timestamp(t; target_tz::VariableTimeZone=tz"America/Los_Angeles") = astimezone(
    ZonedDateTime(Dates.unix2datetime(t), tz"GMT"), target_tz
)

timestamp (generic function with 1 method)

In [4]:
using MySQL
using DataFrames
function get_emeter_data(conn::MySQL.Connection, name::String, idx::Int64)
    DBInterface.execute(
        DBInterface.prepare(conn,
            "SET @v1 := (SELECT emeter_start FROM $(name) WHERE idx = $(idx))"
        )
    )
    DBInterface.execute(
        DBInterface.prepare(conn,
            "SET @v2 := (SELECT emeter_end FROM $(name) WHERE idx = $(idx))"
        )
    )
    events = DBInterface.execute(
        DBInterface.prepare(conn,
            "SELECT * FROM $(name)_emeter WHERE idx BETWEEN @v1 and @v2"
        )
    ) |> DataFrame
    events = transform(
        events,
        :t => (x->timestamp.(x)) => :timestamp
    )
    return events
end

get_emeter_data (generic function with 1 method)

In [5]:
import Base: @kwdef
@kwdef mutable struct DBCredentials
    host::String
    name::String
    username::String
    password::String
end

DBCredentials

In [6]:
using JSON
db_credenitals = DBCredentials(
    ;JSON.parsefile(
        joinpath(data_directory, "db.json"),
        dicttype=Dict{Symbol,String}
    )...
)
# db_credenitals.host = "127.0.0.1";

In [7]:
db_conn = DBInterface.connect(
    MySQL.Connection, db_credenitals.host,
    db_credenitals.username, db_credenitals.password,
    db=db_credenitals.name
)

MySQL.Connection(host="45.33.40.165", user="ccs", port="3306", db="coffee")

In [8]:
name = "Arstotzka_Ministry_of_Energy"
idx_max = 61
idx_blacklist = [
    1,2,3,4,5,6,7,8,9,  # Debugging
    51  #  Battery Charger
]

all_emeter_data = DataFrame[]

for idx=1:idx_max
    if idx in idx_blacklist
        continue
    end
    push!(all_emeter_data, get_emeter_data(db_conn, name, idx))
end

In [9]:
all_emeter_data

51-element Vector{DataFrame}:
 6×7 DataFrame
 Row │ idx    t          ma      mv      mw      wh      timestamp             ⋯
     │ Int32  Float64?   Int32?  Int32?  Int32?  Int32?  ZonedDat…             ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │    12  1.64349e9      59  121266    7139    7512  2022-01-29T12:19:41.3 ⋯
   2 │    13  1.64349e9      59  121266    7139    7512  2022-01-29T12:20:45.1
   3 │    14  1.64349e9      59  121266    7139    7512  2022-01-29T12:20:53.3
   4 │    15  1.64349e9      59  121266    7139    7512  2022-01-29T12:20:54.6
   5 │    16  1.64349e9      59  121266    7139    7512  2022-01-29T12:20:55.5 ⋯
   6 │    17  1.64349e9      59  121266    7139    7512  2022-01-29T12:20:56.5
                                                                1 column omitted
 1×7 DataFrame
 Row │ idx    t          ma      mv      mw      wh      timestamp             ⋯
     │ Int32  Float64?   Int32?  Int32?  Int32?  Int32?  

In [10]:
all_emeter_data = vcat(all_emeter_data...)

,idx,t,ma,mv,mw,wh,timestamp
,Int32,Float64?,Int32?,Int32?,Int32?,Int32?,ZonedDat…
1,12,1.64349e9,59,121266,7139,7512,2022-01-29T12:19:41.336-08:00
2,13,1.64349e9,59,121266,7139,7512,2022-01-29T12:20:45.166-08:00
3,14,1.64349e9,59,121266,7139,7512,2022-01-29T12:20:53.324-08:00
4,15,1.64349e9,59,121266,7139,7512,2022-01-29T12:20:54.646-08:00
5,16,1.64349e9,59,121266,7139,7512,2022-01-29T12:20:55.542-08:00
6,17,1.64349e9,59,121266,7139,7512,2022-01-29T12:20:56.556-08:00
7,18,1.64349e9,59,121266,7139,7512,2022-01-29T12:21:03.581-08:00
8,19,1.64349e9,59,121266,7139,7512,2022-01-29T12:21:16.887-08:00
9,20,1.64349e9,59,121266,7139,7512,2022-01-29T12:21:20.293-08:00


In [11]:
describe(all_emeter_data)

,variable,mean,min,median,max
,Symbol,Union…,Any,Union…,Any
1,idx,2.4833e6,12,2.48353e6,4958709
2,t,1.6465e9,1.64349e9,1.6465e9,1.64955e9
3,ma,414.049,0,60.0,9012
4,mv,1.21104e5,106168,121184.0,124168
5,mw,45782.7,6466,7246.0,918210
6,wh,14888.4,0,12321.0,37943
7,timestamp,,2022-01-29T12:19:41.336-08:00,,2022-04-09T17:29:25.218-07:00
